# Fine-Tuning a Model for Music Classification

## The Dataset

In [2]:
from datasets import load_dataset

gtzan = load_dataset(
    "marsyas/gtzan",
    "all",
    trust_remote_code=True
)
gtzan

Generating train split: 999 examples [00:00, 2576.30 examples/s]


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [3]:
gtzan = gtzan["train"].train_test_split(
    test_size=0.1,
    shuffle=True,
    seed=42
)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [4]:
gtzan["train"][0]

{'file': 'C:\\Users\\placasa\\.cache\\huggingface\\datasets\\downloads\\extracted\\02b4528494d8d7cd9813e467a5bd9196e7d94a84c480e9168508a031ff2be9fb\\genres\\pop\\pop.00098.wav',
 'audio': {'path': 'C:\\Users\\placasa\\.cache\\huggingface\\datasets\\downloads\\extracted\\02b4528494d8d7cd9813e467a5bd9196e7d94a84c480e9168508a031ff2be9fb\\genres\\pop\\pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229]),
  'sampling_rate': 22050},
 'genre': 7}

In [5]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

'pop'

In [6]:
import gradio as gr

def generate_audio():
    example = gtzan["train"].shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"]
    ), id2label_fn(example["genre"])

with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(4):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

demo.launch(debug=True)

c:\Users\placasa\AppData\Local\miniconda3\envs\huggingface_audio_course_py311\Lib\site-packages\gradio\processing_utils.py:741: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


## Preprocessing the data

In [7]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id,
    do_normalize=True,
    return_attention_mask=True
)

c:\Users\placasa\AppData\Local\miniconda3\envs\huggingface_audio_course_py311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\placasa\.cache\huggingface\hub\models--ntu-spml--distilhubert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [9]:
from datasets import Audio

gtzan = gtzan.cast_column(
    "audio",
    Audio(sampling_rate=sampling_rate)
)

In [10]:
gtzan["train"][0]

{'file': 'C:\\Users\\placasa\\.cache\\huggingface\\datasets\\downloads\\extracted\\02b4528494d8d7cd9813e467a5bd9196e7d94a84c480e9168508a031ff2be9fb\\genres\\pop\\pop.00098.wav',
 'audio': {'path': 'C:\\Users\\placasa\\.cache\\huggingface\\datasets\\downloads\\extracted\\02b4528494d8d7cd9813e467a5bd9196e7d94a84c480e9168508a031ff2be9fb\\genres\\pop\\pop.00098.wav',
  'array': array([ 0.08735093,  0.20183387,  0.47908676, ..., -0.1874318 ,
         -0.23294398, -0.13517427]),
  'sampling_rate': 16000},
 'genre': 7}

In [12]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


In [13]:
inputs = feature_extractor(
    sample["array"],
    sampling_rate=sample["sampling_rate"]
)

print(f"Inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, "
    f"Variance: {np.var(inputs['input_values']):.3}"
)

Inputs keys: ['input_values', 'attention_mask']
Mean: -7.03e-09, Variance: 1.0


In [14]:
max_duration = 30

In [15]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True
    )

    return inputs

In [16]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=8,
    num_proc=1
)
gtzan_encoded

Map: 100%|██████████| 100/100 [00:04<00:00, 20.10 examples/s]


DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [17]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

In [18]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

## Fine-tuning the model

In [19]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False
)

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """
    Computes accuracy on a batch of predictions.
    """
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(
        predictions=predictions,
        references=eval_pred.label_ids
    )

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification"
}

## Share model

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "audio-classification",
    model="sanchit-gandhi/distilhubert-finetuned-gtzan"
)